<table class="tfo-notebook-buttons" align="center">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/practicaldl/Practical-Deep-Learning-Book/blob/master/code/chapter-4/1-feature-extraction.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/practicaldl/Practical-Deep-Learning-Book/blob/master/code/chapter-4/1-feature-extraction.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
  </td>
</table>


This code is part of [Chapter 4 - Building a Reverse Image Search Engine: Understanding Embeddings](https://learning.oreilly.com/library/view/practical-deep-learning/9781492034858/ch04.html).

Note: In order to run this notebook on Google Colab you need to [follow these instructions](https://colab.research.google.com/github/googlecolab/colabtools/blob/master/notebooks/colab-github-demo.ipynb#scrollTo=WzIRIt9d2huC) so that the local data such as the images are available in your Google Drive.

Note 7/24/2021: This notebook has been updated as of 7/24/2021.

In [1]:
try:
    # Mount Google Drive
    from google.colab import drive

    drive.mount("/content/gdrive")

    IS_COLAB_ENV = True
except:
    IS_COLAB_ENV = False
IS_COLAB_ENV

Mounted at /content/gdrive


True

# Feature Extraction

This notebook is the first among six of the follow along Jupyter Notebook for Chapter 4. We will extract features from pretrained models like VGG-16, VGG-19, ResNet-50, InceptionV3 and MobileNet and benchmark them using the Caltech101 dataset.

## Dataset:

In the `data` directory of the repo, download the Caltech101 dataset (or try it on your dataset). Please note (as of 01 September 2020) the Caltech 101 dataset has moved locations and now has to be downloaded through Google Drive using `gdown`.

```
$ gdown https://drive.google.com/uc?id=137RyRjvTBkBiIfeYBNZBtViDHQ6_Ewsp --output caltech101.tar.gz

$ tar -xvzf caltech101.tar.gz

$ mv 101_ObjectCategories datasets/caltech101
```
Note that there is a 102nd category called ‘BACKGROUND_Google’ consisting of random images not contained in the first 101 categories, which needs to be deleted before we start experimenting.

```
$ rm -rf datasets/caltech101/BACKGROUND_Google
```

In [2]:
!mkdir -p ../../datasets
!pip install gdown
!gdown https://drive.google.com/uc?id=137RyRjvTBkBiIfeYBNZBtViDHQ6_Ewsp --output ../../datasets/caltech101.tar.gz
!tar -xvzf ../../datasets/caltech101.tar.gz --directory ../../datasets
!mv ../../datasets/101_ObjectCategories ../../datasets/caltech101
!rm -rf ../../datasets/caltech101/BACKGROUND_Google

Streaming output truncated to the last 5000 lines.
101_ObjectCategories/chair/image_0005.jpg
101_ObjectCategories/chair/image_0006.jpg
101_ObjectCategories/chair/image_0007.jpg
101_ObjectCategories/chair/image_0008.jpg
101_ObjectCategories/chair/image_0010.jpg
101_ObjectCategories/chair/image_0011.jpg
101_ObjectCategories/chair/image_0012.jpg
101_ObjectCategories/chair/image_0013.jpg
101_ObjectCategories/chair/image_0014.jpg
101_ObjectCategories/chair/image_0016.jpg
101_ObjectCategories/chair/image_0017.jpg
101_ObjectCategories/chair/image_0018.jpg
101_ObjectCategories/chair/image_0019.jpg
101_ObjectCategories/chair/image_0020.jpg
101_ObjectCategories/chair/image_0022.jpg
101_ObjectCategories/chair/image_0023.jpg
101_ObjectCategories/chair/image_0024.jpg
101_ObjectCategories/chair/image_0025.jpg
101_ObjectCategories/chair/image_0026.jpg
101_ObjectCategories/chair/image_0028.jpg
101_ObjectCategories/chair/image_0029.jpg
101_ObjectCategories/chair/image_0030.jpg
101_ObjectCategories/chai

In [3]:
import numpy as np
from numpy.linalg import norm
import pickle
from tqdm import tqdm
from tqdm.notebook import tqdm as tqdm_notebook
import os
import random
import time
import math
import tensorflow
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.vgg19 import VGG19
from tensorflow.keras.applications.mobilenet import MobileNet
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.applications.xception import Xception
from tensorflow.keras.models import Model
from tensorflow.keras.layers import (
    Input,
    Flatten,
    Dense,
    Dropout,
    GlobalAveragePooling2D,
)

We will define a helper function that allows us to choose any pretrained model with all the necessary details for our experiments.

In [4]:
def model_picker(name):
    if name == "vgg16":
        model = VGG16(
            weights="imagenet",
            include_top=False,
            input_shape=(224, 224, 3),
            pooling="max",
        )
    elif name == "vgg19":
        model = VGG19(
            weights="imagenet",
            include_top=False,
            input_shape=(224, 224, 3),
            pooling="max",
        )
    elif name == "mobilenet":
        model = MobileNet(
            weights="imagenet",
            include_top=False,
            input_shape=(224, 224, 3),
            pooling="max",
            depth_multiplier=1,
            alpha=1,
        )
    elif name == "inception":
        model = InceptionV3(
            weights="imagenet",
            include_top=False,
            input_shape=(224, 224, 3),
            pooling="max",
        )
    elif name == "resnet":
        model = ResNet50(
            weights="imagenet",
            include_top=False,
            input_shape=(224, 224, 3),
            pooling="max",
        )
    elif name == "xception":
        model = Xception(
            weights="imagenet",
            include_top=False,
            input_shape=(224, 224, 3),
            pooling="max",
        )
    else:
        print("Specified model not available")
    return model

Now, let's put our function to use.

In [5]:
model_architecture = "resnet"
model = model_picker(model_architecture)

94765736/94765736 [==============================] - 1s 0us/step


Let's define a function to extract image features given an image and a model. We developed a similar function in Chapter-2

In [6]:
def extract_features(img_path, model):
    input_shape = (224, 224, 3)
    img = image.load_img(img_path, target_size=(input_shape[0], input_shape[1]))
    img_array = image.img_to_array(img)
    expanded_img_array = np.expand_dims(img_array, axis=0)
    preprocessed_img = preprocess_input(expanded_img_array)
    features = model.predict(preprocessed_img)
    flattened_features = features.flatten()
    normalized_features = flattened_features / norm(flattened_features)
    return normalized_features

Let's see the feature length the model generates by running on an example image. If you don't have the usual cat image available locally, let's download it!

In [7]:
def download_sample_image(filename):
    import requests

    url = f"https://raw.githubusercontent.com/PracticalDL/Practical-Deep-Learning-Book/master/sample-images/{filename}"
    open(filename, "wb").write(requests.get(url).content)

In [8]:
IMG_PATH = "../../sample-images/cat.jpg"
if IS_COLAB_ENV:
    IMG_PATH = "cat.jpg"
    download_sample_image(IMG_PATH)

In [9]:
features = extract_features(IMG_PATH, model)
print("Total length of features for one image: ", len(features))

1/1 [==============================] - 9s 9s/step
Total length of features for one image:  2048


Now, we will see how much time it takes to extract features of one image. For the cat image, it should ideally be around 81 ms. Depending on your image size, the final time may be different and it depends on the time to read, resize, preprocess, and extract features.

In [10]:
%timeit features = extract_features('cat.jpg', model)

1/1 [==============================] - 0s 26ms/step
89.3 ms ± 1.4 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


## Benchmarking time taken to extract features over the entire dataset

The time taken to extract features is dependent on a few factors such as image size, computing power etc. A better benchmark would be running the network over an entire dataset. A simple change to the existing code will allow this.

Let's make a handy function to recursively get all the image files under a root directory.

In [11]:
extensions = [".jpg", ".JPG", ".jpeg", ".JPEG", ".png", ".PNG"]


def get_file_list(root_dir):
    file_list = []
    for root, directories, filenames in os.walk(root_dir):
        for filename in filenames:
            if any(ext in filename for ext in extensions):
                filepath = os.path.join(root, filename)
                if os.path.exists(filepath):
                    file_list.append(filepath)
                else:
                    print(filepath)
    return file_list

Now, let's run the extraction over the entire dataset and time it.

In [12]:
# path to the your datasets
root_dir = "../../datasets/caltech101"
filenames = sorted(get_file_list(root_dir))
print(len(filenames))

8677


In [13]:
standard_feature_list = []
for i in tqdm_notebook(range(len(filenames))):
    standard_feature_list.append(extract_features(filenames[i], model))

  0%|          | 0/8677 [00:00<?, ?it/s]

Streaming output truncated to the last 5000 lines.
1/1 [==============================] - 0s 34ms/step


Now let's try the same with the Keras Image Generator functions.

In [14]:
batch_size = 128
datagen = tensorflow.keras.preprocessing.image.ImageDataGenerator(
    preprocessing_function=preprocess_input
)

generator = datagen.flow_from_directory(
    root_dir,
    target_size=(224, 224),
    class_mode=None,
    batch_size=batch_size,
    shuffle=False,
)

num_images = len(generator.filenames)
num_epochs = int(math.ceil(num_images / batch_size))

start_time = time.time()
feature_list = []
feature_list = model.predict(generator, num_epochs)
end_time = time.time()

Found 8677 images belonging to 101 classes.
68/68 [==============================] - 33s 414ms/step


In [15]:
for i, features in enumerate(feature_list):
    feature_list[i] = features / norm(features)

feature_list = feature_list.reshape(len(feature_list), -1)

print("Num images   = ", len(generator.classes))
print("Shape of feature_list = ", feature_list.shape)
print("Time taken in sec = ", end_time - start_time)

Num images   =  8677
Shape of feature_list =  (8677, 2048)
Time taken in sec =  33.28644275665283


By now, we have generated features from the entire dataset of images, using two different methods. We experimented with our own code of extracting features and compared it against the generator functionality in Keras. Below you'll see some summaries of experiments we ran, by varying the batch size parameter.

### GPU Utilization's effect on time taken by varying batch size


GPUs are optimized to parallelize the feature generation process and hence will give better results when multiple images are passed instead of just one image.
The opportunity to improve can be seen based on GPU Utilization. Low GPU Utilization indicates an opportunity to further improve the througput.


GPU Utilization can be seen using the nvidia-smi command. To update it every half a second

    watch -n .5 nvidia-smi
    
To pool the GPU utilization every second and dump into a file

    nvidia-smi --query-gpu=utilization.gpu --format=csv,noheader,nounits -f gpu_utilization.csv -l 1
    
To calculate median GPU Utilization from the file generated

    sort -n gpu_utilization.csv | datamash median 1

|Model |Time second (sec) | batch_size | % GPU Utilization | Implementation|
|-|-|-|-|-|
|Resnet50 | 124  | 1  | 52 | extract_features    |
|Resnet50 | 98   | 1  | 72 | ImageDataGenerator |
|Resnet50 | 57   | 2  | 81 | ImageDataGenerator |
|Resnet50 | 40   | 4  | 88 | ImageDataGenerator |
|Resnet50 | 34   | 8  | 94 | ImageDataGenerator |
|Resnet50 | 29   | 16 | 97 | ImageDataGenerator |
|Resnet50 | 28   | 32 | 97 | ImageDataGenerator |
|Resnet50 | 28   | 64 | 98 | ImageDataGenerator |

### Some benchmarks on different model architectures to see relative speeds

Keeping batch size of 64, benchmarking the different models

|Model |items/second |
|-|-|
| VGG19     | 31.06 |
| VGG16     | 28.16 |
| Resnet50  | 28.48 |
| Inception | 20.07 |
| Mobilenet | 13.45 |

With the benchmarking experiments squared away, let's save the features as intermediate files to use later.

In [16]:
filenames = [root_dir + "/" + s for s in generator.filenames]

In [17]:
generator_classes_path = "data/class_ids-caltech101.pickle"
filenames_path = "data/filenames-caltech101.pickle"
model_features_path = f"data/features-caltech101-{model_architecture}.pickle"

if IS_COLAB_ENV:
    os.makedirs(
        "/content/gdrive/MyDrive/Practical-Deep-Learning-Book/code-outputs/chapter-4/data/",
        exist_ok=True,
    )
    generator_classes_path = f"/content/gdrive/MyDrive/Practical-Deep-Learning-Book/code-outputs/chapter-4/{generator_classes_path}"
    filenames_path = f"/content/gdrive/MyDrive/Practical-Deep-Learning-Book/code-outputs/chapter-4/{filenames_path}"
    model_features_path = f"/content/gdrive/MyDrive/Practical-Deep-Learning-Book/code-outputs/chapter-4/{model_features_path}"
else:
    !mkdir data

pickle.dump(generator.classes, open(generator_classes_path, "wb"))
pickle.dump(filenames, open(filenames_path, "wb"))
pickle.dump(feature_list, open(model_features_path, "wb"))

Let's train a finetuned model as well and save the features for that as well.

In [18]:
TRAIN_SAMPLES = 8677
NUM_CLASSES = 101
IMG_WIDTH, IMG_HEIGHT = 224, 224

In [19]:
train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    zoom_range=0.2,
)

In [20]:
train_generator = train_datagen.flow_from_directory(
    root_dir,
    target_size=(IMG_WIDTH, IMG_HEIGHT),
    shuffle=True,
    seed=12345,
    class_mode="categorical",
)

Found 8677 images belonging to 101 classes.


In [21]:
def model_maker():
    base_model = ResNet50(include_top=False, input_shape=(IMG_WIDTH, IMG_HEIGHT, 3))
    for layer in base_model.layers[:]:
        layer.trainable = False
    input = Input(shape=(IMG_WIDTH, IMG_HEIGHT, 3))
    custom_model = base_model(input)
    custom_model = GlobalAveragePooling2D()(custom_model)
    custom_model = Dense(64, activation="relu")(custom_model)
    custom_model = Dropout(0.5)(custom_model)
    predictions = Dense(NUM_CLASSES, activation="softmax")(custom_model)
    return Model(inputs=input, outputs=predictions)

In [22]:
model_finetuned = model_maker()
model_finetuned.compile(
    loss="categorical_crossentropy",
    optimizer=tensorflow.keras.optimizers.Adam(0.001),
    metrics=["acc"],
)
model_finetuned.fit(
    train_generator,
    steps_per_epoch=math.ceil(float(TRAIN_SAMPLES) / batch_size),
    epochs=10,
)

Epoch 1/10
68/68 [==============================] - 35s 415ms/step - loss: 3.7974 - acc: 0.2142
Epoch 2/10
68/68 [==============================] - 29s 420ms/step - loss: 2.8548 - acc: 0.3580
Epoch 3/10
68/68 [==============================] - 29s 421ms/step - loss: 2.4591 - acc: 0.4274
Epoch 4/10
68/68 [==============================] - 29s 419ms/step - loss: 2.0874 - acc: 0.4936
Epoch 5/10
68/68 [==============================] - 29s 423ms/step - loss: 1.8255 - acc: 0.5487
Epoch 6/10
68/68 [==============================] - 29s 421ms/step - loss: 1.6586 - acc: 0.5722
Epoch 7/10
68/68 [==============================] - 29s 423ms/step - loss: 1.5687 - acc: 0.5966
Epoch 8/10
68/68 [==============================] - 29s 420ms/step - loss: 1.5300 - acc: 0.5910
Epoch 9/10
68/68 [==============================] - 29s 421ms/step - loss: 1.4658 - acc: 0.6025
Epoch 10/10
68/68 [==============================] - 29s 418ms/step - loss: 1.3404 - acc: 0.6356


In [23]:
model_path = "data/model-finetuned.h5"
if IS_COLAB_ENV:
    model_path = f"/content/gdrive/MyDrive/Practical-Deep-Learning-Book/code-outputs/chapter-4/{model_path}"

model_finetuned.save(model_path)

In [25]:
start_time = time.time()
feature_list_finetuned = []
feature_list_finetuned = model_finetuned.predict(generator, num_epochs)
end_time = time.time()

for i, features_finetuned in enumerate(feature_list_finetuned):
    feature_list_finetuned[i] = features_finetuned / norm(features_finetuned)

feature_list = feature_list_finetuned.reshape(len(feature_list_finetuned), -1)

print("Num images   = ", len(feature_list_finetuned))
print("Shape of feature_list = ", feature_list.shape)
print("Time taken in sec = ", end_time - start_time)

68/68 [==============================] - 30s 448ms/step
Num images   =  8677
Shape of feature_list =  (8677, 101)
Time taken in sec =  41.43211245536804


In [26]:
finetuned_model_features_path = "data/features-caltech101-resnet-finetuned.pickle"
if IS_COLAB_ENV:
    finetuned_model_features_path = f"/content/gdrive/MyDrive/Practical-Deep-Learning-Book/code-outputs/chapter-4/{finetuned_model_features_path}"

pickle.dump(feature_list, open(finetuned_model_features_path, "wb"))